In [20]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("ashishjangra27/face-mask-12k-images-dataset")

# print("Path to dataset files:", path)

In [21]:
# import os
# print("There dictoryes in this file",os.listdir(path))

In [22]:
# image_dir=os.listdir(path)

In [23]:
# base_dir=os.path.join(path,'image_dir')

In [24]:
# base_dir

Make Face Mask Detection Project Using CNN(Convluation Neural Network)

In [38]:
# %pip install scikit-learn
# %pip install kagglehub
# %pip install kaggle
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install seaborn
# %pip install tensorflow
# %pip install keras
# %pip install sklearn
# %pip install opencv-python

2. Load Libraries

In [36]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


3. Set parameters

In [39]:
IMG_SIZE=224
# VGG ke liye standard size
data=[]
lable=[]

4. Read & Resize Images

In [ ]:
# # 

# # dataset path
# base_path = "dataset"

# categories = ["with_mask", "without_mask"]

# for category in categories:
#     path = os.path.join(base_path, category)
#     label = categories.index(category)   # with_mask -> 0, without_mask -> 1
    
#     for img_name in os.listdir(path):
#         img_path = os.path.join(path, img_name)
#         img = cv2.imread(img_path)
        
#         if img is not None:
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # BGR -> RGB
#             img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # resize
#             data.append(img)
#             labels.append(label)

# data = np.array(data) / 255.0   # normalization (0–1 range)
# labels = np.array(labels)



In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     data, labels, test_size=0.2, random_state=42, stratify=labels
# )

# print("Training data:", X_train.shape)
# print("Testing data:", X_test.shape)


In [62]:
# I have Ornize form of the data

from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMG_SIZE=224
BATCH_SIZE=32
# Data Augmentation for training (rotation, zoom, etc.)

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

# Validation & Test data ke liye sirf rescale
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)




👉 Why?

rescale=1./255 → Images ke pixel values (0–255) ko normalize karke (0–1) range mein le aata hai. Neural nets ke liye normalization zaruri hai.

rotation_range=20 → Images ko randomly 20° tak rotate karta hai → model rotation ke against robust ho jata hai.

zoom_range=0.2 → Random zoom apply hota hai → model zoomed faces pe bhi sahi predict karega.

shear_range=0.2 → Random slant/shear effect deta hai → different face angles ke liye useful.

horizontal_flip=True → Left-right mirror images banata hai → model left profile aur right profile dono pe kaam kare.

💡 Train dataset mein augmentation isliye hoti hai, kyunki hume model ko zyada variations dikhane hain.

Training Generator

In [63]:
train_Generator=train_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Train",
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=32,
    class_mode="binary"
)

Found 10000 images belonging to 2 classes.


In [66]:
#Validation Generator

from random import shuffle


val_Generator = train_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Validation",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "binary"
)

#

test_Generator = test_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Test",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "binary",
    shuffle = False
)

Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


🔹 Option 1: Simple CNN (From Scratch)

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # 1st Convolution Layer
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(pool_size=(2,2)),
    
    # 2nd Convolution Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    # 3rd Convolution Layer
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    # Flatten & Fully Connected Layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')   # Binary classification (mask/no mask)
])

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


d:\Project-to-learn\.machine\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_Generator,
    validation_data=val_Generator,
    epochs=10
)

model.save('face_mask_detection_model.h5')


d:\Project-to-learn\.machine\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 239s 760ms/step - accuracy: 0.8326 - loss: 0.3984 - val_accuracy: 0.9475 - val_loss: 0.1628
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 130s 417ms/step - accuracy: 0.9295 - loss: 0.1951 - val_accuracy: 0.9400 - val_loss: 0.1611
Epoch 3/10
289/313 ━━━━━━━━━━━━━━━━━━━━ 9s 378ms/step - accuracy: 0.9426 - loss: 0.1587